In [1]:
using PyPlot

INFO: Loading help data...


In [81]:
# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

fig = plt.figure()
ax=fig[:add_subplot](111)

x=[1:1:100]
y=[rand(100)]

li=ax[:plot](x, y)[1]

fig[:canvas][:draw]()
plt.show()

y2=[rand(100,100)]

@time for i=1:100
    li[:set_ydata](y2[i,:])
    fig[:canvas][:blit](ax[:bbox])
    #fig[:canvas][:draw]()
end


elapsed time: 5.147000875 seconds (593200 bytes allocated)
